In [1]:
import os
from wordcloud import WordCloud
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/ebrahim/Downloads/spark-3.0.1-bin-hadoop3.2"

In [2]:
import json
import os
import findspark
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window
findspark.init()
os.environ["PYSPARK_DRIVER_PYTHON"] = os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.7"


In [54]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com','translate.google.com','translate.google.co.kr'])

In [53]:
translator = Translator(service_urls=['translate.googleapis.com','translate.google.com','translate.google.co.kr'])

In [5]:
# findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Read profiles

In [8]:
profiles_df = spark.read.parquet("./cleaned_ids/parquet/*.parquet").filter(F.col("description").isNotNull()).filter(F.col("description") != '').select(
    F.col("id"),
    F.col("description"),
).cache()

In [9]:
profiles_df.select(F.col("id")).distinct().count()

15183

In [77]:
rows = profiles_df.collect()

In [75]:
skipped_indexes = set()
translated_rows = 0
english_rows = 0
none_rows = 0

In [76]:
translated_description = {} 

In [65]:
translator.translate('svízelná situace', src='cs', dest='en').text

'difficult situation'

In [ ]:
for i in range(len(rows)):
    row = rows[i]
    if row.id in translated_description:
        continue
    try:
        description = row.description
        if not description:
            none_rows += 1
            translated_description[row.id] = description
            continue
        translated_description[row.id] = translator.translate(description, dest='en').text
        translated_rows += 1
        if translated_rows % 100 == 0:
            print("Done translating 100 records", len(translated_description))
    except Exception as e:
        print("Catch", e)
        print("For index", i)
        skipped_indexes.add(i)

In [80]:
translated_rows

15183

In [81]:
translated_df = spark.createDataFrame(
    [
        [k, v] for k, v in translated_description.items()
        
    ], schema=T.StructType([
        T.StructField("id", T.IntegerType(), True),
        T.StructField("translated_description", T.StringType(), True),
    ])
).cache()

In [82]:
translated_df.count()

15183

In [153]:
translated_df.withColumn(
    "translated_description",
    F.regexp_replace(F.col("translated_description"), r"\r", " ")
).withColumn(
    "translated_description",
    F.regexp_replace(F.col("translated_description"), r"\n", " ")
).repartition(1).write.csv("./translated/csv_3", header=True, mode="overwrite", sep=',')

> Write to csv in order to read the data and translate the remaining rows with goggle sheet because the python package couldn't manage to translate everything

In [84]:
all_profiles_df = spark.read.parquet("./cleaned_ids/parquet/*.parquet").cache()

In [156]:
translated_gsheet_df = spark.read.csv("./translated/gsheet_translation.csv", header=True).select(
    F.col("id"),
    F.col("translated_gsheet").alias("translated_description")
)

> Read tramslated data from google sheet

In [158]:
translated_gsheet_df.show(2)

+---+----------------------+
| id|translated_description|
+---+----------------------+
|  1|  "in short, I'm a ...|
|  2|  lets see, i am sp...|
+---+----------------------+
only showing top 2 rows



In [159]:
all_profiles_df.join(translated_gsheet_df, on=["id"], how="left").withColumn(
    "translated_description",
    F.when(
        F.col("translated_description").isNull(),
        F.col("description"),
    ).otherwise(F.col("translated_description"))
).write.parquet("./translated/gsheet")

In [94]:
spark.stop()